In [6]:
import qiskit as qiskit
import numpy as np
from random import randint
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit import IBMQ
IBMQ.save_account('7e245f54848bdbcc6bedd42fcafcd2fbe8f81b765b2537e32d39f812c3ccc2e9c755a6ac3e3edc7529982f02954bff4b84cba76cef7fe71928b9f01b092feedf')
IBMQ.load_account()
from qiskit.providers.aer.noise import NoiseModel
import copy

configrc.store_credentials:WARNING:2020-06-26 20:14:09,743: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2020-06-26 20:14:10,038: Credentials are already in use. The existing account in the session will be replaced.


# Creating Random Circuits

We want to create random circuits to train and validate our neural network on. To do this, we utilize random number generators with a limit on the circuit depth of the largest quantum computer available to us from the IBMQ Experience, 15 qubits. We want two things from this function:
1. Output a QuantumCircuit object that we can later run on multiple backends
2. Output the data in a way that our neural network can understand:
    * A list which has the following Dimensions: number of qubits x circuit depth
    * Each element in the list will correspond to a gate given by the dictionary `operators`

In [34]:
def create_circuit():
    """This function takes no inputs and outputs a random circuit in neural network representation 
    as well as a quantum circuit object"""
    num_qubits = randint(2,5) #Set the circuit width
    len_circuit = randint(10,30) #Set the circuit depth
    circ = np.zeros((num_qubits,len_circuit)) #Initialize the circuit representation for the Neural Net
    qc = qiskit.QuantumCircuit(num_qubits) #Initialize the actual quantum circuit
    operators = {0:qc.id,1:qc.x,2:qc.y,3:qc.z,4:qc.h,5:qc.cx,6:qc.swap} #Define the operators and their corresponding Neural Net representations
    for i in range(len_circuit):
        ctrl = randint(0,num_qubits-1) #Choose a qubit for a single qubit gat to act on or a ctrl qubit for a 2 qubit gate to act on
        targ = randint(0,num_qubits-1) #Choose a target qubit for a 2 qubit gate to act on
        gate_num = randint(1,6) #Choose a gate to implement
        while ctrl == targ: #Make sure the target and ctrl qubit are different
            targ = randint(0,num_qubits-1)
        try:
            #Implement the gate if it is a single qubit gate and add it to the neural net representation
            operators.get(gate_num)(ctrl) 
            circ[ctrl][i] = gate_num
        except:
            try:
                #Implement the gate if it is a 2 qubit gate and add it to the neural net representation
                operators.get(gate_num)(ctrl,targ)
                circ[ctrl][i] = gate_num
                circ[targ][i] = gate_num
            except:
                gate_num = randint(1,6)
    qc.measure_all()
    return circ, qc

In [35]:
def list_of_circuits(num_of_circuits):
    circuits = [0]*num_of_circuits #initialize list by number of desired circuits
    circuit_arrays = [0]*num_of_circuits #initialize list by number of desired circuits

    for i in range(num_of_circuits): 
        #calls create_circuits function desired number of times and puts objects into the arrays
        circuit_arrays[i], circuits[i] = create_circuit()
    return circuit_arrays, circuits

# Creating the Validation Answer Key

Here we use the KL divergence to analyze which probability distribution obtained from running our circuit on multiple different backends is the closest to the circuit result obtained from running on a simulator with no noise. We save this data in a list of lists, where each row can be thought of as referencing a circuit and each column can be thought of as referencing a backend. 

In [36]:
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

In [ ]:
circuit_arrays, circuits = list_of_circuits(1)
list_of_backends = IBMQ.get_provider('ibm-q').backends()
list_of_backends.remove(IBMQ.get_provider('ibm-q').get_backend('ibmq_qasm_simulator'))
simulator = Aer.get_backend('qasm_simulator')

validation_set = np.zeros((len(circuits),len(list_of_backends)))
shots = 10000
for i in range(len(circuits)):
    previous_divergence = 1
    best_backend = 0;
    for j in range(len(list_of_backends)):
        if np.shape(circuit_arrays[i])[0] < list_of_backends[j].configuration().num_qubits:
            coupling_map = list_of_backends[j].configuration().coupling_map
            basis_gates = list_of_backends[j].configuration().basis_gates
            noise_model = NoiseModel.from_backend(list_of_backends[j])
            psi_0 = execute(circuits[i], simulator, shots = shots, coupling_map = coupling_map, basis_gates = basis_gates).result().get_counts()
            psi_1 = execute(circuits[i], simulator, shots = shots, coupling_map = coupling_map, basis_gates = basis_gates, noise_model = noise_model).result().get_counts()
            
            psi_00 = copy.deepcopy(psi_1)
            for bit in psi_0.keys():
                psi_00[bit] = psi_0.get(bit)
            
            
            psi_0 = np.asarray([value/shots for value in psi_00.values()])
            psi_1 = np.asarray([value/shots for value in psi_1.values()])
            divergence = np.abs(kl_divergence(psi_0, psi_1))
            print(divergence)
            if divergence <  previous_divergence:
                best_backend = j
                previous_divergence = divergence
                print(previous_divergence)
    validation_set[i, best_backend] = 1
        
print("done")       
        

0.11596507701514297
0.11596507701514297
0.13465226510042266
0.12078624303644254
0.08032627178189028
0.08032627178189028
0.13178303559179358
0.1626901898877588
0.1193561408394521
0.11107771799628681
done


In [38]:
validation_set

array([[0., 0., 0., 1., 0., 0., 0., 0., 0.]])